In [2]:
data_dir = "dataset/cropped"


In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.version)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: <module 'tensorflow._api.v2.version' from 'c:\\Users\\ASUS\\anaconda3\\envs\\tf_gpu\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>
Num GPUs Available: 1


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import os

# Parameters
DATA_DIR = "dataset/cropped"   # Your dataset root directory
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS_INITIAL = 10
EPOCHS_FINE_TUNE = 10

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Save class names
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# 1. Preprocess datasets with MobileNetV2 preprocessing and prefetch for performance
def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).prefetch(buffer_size=tf.data.AUTOTUNE)

# 2. Load MobileNetV2 base model without top layers
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet',
                         pooling='avg')

# 3. Freeze the entire base model initially
base_model.trainable = False

#Data Augmentation

data_augmentation = models.Sequential([
    layers.RandomFlip('horizontal', input_shape=(224, 224, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(factor=0.2),
    layers.RandomContrast(factor=0.2),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomSaturation(factor=0.2),
])

# 4. Add classification head
model = models.Sequential([
    data_augmentation,
    layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),  # Explicit input shape
    base_model,
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

# 5. Compile model for initial training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Initial training (feature extraction)
print("Starting initial training...")
model.fit(train_ds,
          validation_data=val_ds,
          epochs=EPOCHS_INITIAL)

# 7. Fine-tuning: unfreeze last 10 layers of base model
print("Unfreezing last 10 layers of base model for fine-tuning...")

# Unfreeze the base model first
base_model.trainable = True

# Freeze all layers except the last 10 layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Recompile with lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 8. Fine-tune the model
print("Starting fine-tuning...")
model.fit(train_ds,
          validation_data=val_ds,
          epochs=EPOCHS_FINE_TUNE)

# 9. Save the trained model
model.save("mobilenetv2_face_recognition.h5")
print("Model saved as mobilenetv2_face_recognition.h5")


Found 44982 files belonging to 140 classes.
Using 35986 files for training.
Found 44982 files belonging to 140 classes.
Using 8996 files for validation.
Classes: ['n000002', 'n000003', 'n000004', 'n000005', 'n000006', 'n000007', 'n000008', 'n000010', 'n000011', 'n000012', 'n000013', 'n000014', 'n000015', 'n000016', 'n000017', 'n000018', 'n000019', 'n000020', 'n000021', 'n000022', 'n000023', 'n000024', 'n000025', 'n000026', 'n000027', 'n000028', 'n000030', 'n000031', 'n000032', 'n000033', 'n000034', 'n000035', 'n000036', 'n000037', 'n000038', 'n000039', 'n000041', 'n000042', 'n000043', 'n000044', 'n000045', 'n000046', 'n000047', 'n000048', 'n000049', 'n000050', 'n000051', 'n000052', 'n000053', 'n000054', 'n000055', 'n000056', 'n000057', 'n000058', 'n000059', 'n000060', 'n000061', 'n000062', 'n000063', 'n000064', 'n000065', 'n000066', 'n000067', 'n000068', 'n000069', 'n000070', 'n000071', 'n000072', 'n000073', 'n000074', 'n000075', 'n000076', 'n000077', 'n000079', 'n000080', 'n000081', '

KeyboardInterrupt: 

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9296157450046928726
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1733715559
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11753886590560668989
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


AttributeError: 'PrefetchDataset' object has no attribute 'class_names'